In [6]:
import os
os.environ["CORENLP_HOME"] = r'C:\stanford-corenlp-full-2018-10-05'

import corenlp 
client = corenlp.CoreNLPClient()

def get_nouns(sentence, chunk = False):
    text = sentence.lower()
    res = []
    try:
        pattern = '[{pos:NN}] | [{pos:NNS}] | [{pos:NNP}]'
        matches = client.tokensregex(sentence, pattern)
        response = matches['sentences'][0]
        len_chunk = response['length']
        if len_chunk > 0:
            for index in range(0, len_chunk):
                res.append(response[str(index)]['text'])
    except:
        print('err : ', sentence)
        res = []
    return res

def get_adverbs(sentence, chunk = False):
    text = sentence.lower()
    res = []
    try:
        pattern = '[{pos:RB}]'
        matches = client.tokensregex(sentence, pattern)
        response = matches['sentences'][0]
        len_chunk = response['length']
        if len_chunk > 0:
            for index in range(0, len_chunk):
                res.append(response[str(index)]['text'])
    except:
        print('err')
        res = []
    return res

In [5]:
import pandas as pd
from tqdm import tqdm_notebook

def run(tipe):
    df = pd.read_csv("Results/Aspect Terms Extraction/"+ tipe +".csv")
    sf = pd.DataFrame(columns = ['id','review','category','term'])
    count = 0
    for index in tqdm_notebook(range(len(df))):
        adverbs = []#get_adverbs(df['review'][index])
        new_term = []
        if type(df['term'][index]) != float:
            terms = df['term'][index].split('|')
            for term in terms:
                if term not in adverbs:
                    new_term.append(term)
        
#         for nn in get_nouns(df['review'][index]):
#             new_term.append(nn)
        if len(new_term) == 0:
            new_term = '|'.join(get_nouns(df['review'][index]))
            count += 1
        
        sf = sf.append({'id': df['id'][index], 
                    'review': df['review'][index],
                    'category': df['category'][index],
                    'term': '|'.join(new_term)
                   }, ignore_index=True)
    print(count)

    sf.to_csv("Results/add_noun/"+ tipe +"-partial.csv")
    sf.to_excel("Results/add_noun/"+ tipe +"-partial.xlsx")

In [6]:
run("20")

HBox(children=(IntProgress(value=0, max=699), HTML(value='')))

err
err

191


In [4]:
import pandas as pd
from tqdm import tqdm_notebook

def chunk(filename):
    df = pd.read_csv(filename)
    sf = pd.DataFrame(columns = ['id','review', 'target', 'category','term'])
    for index in tqdm_notebook(range(len(df))):
        new_term = []
        nouns = get_nouns(df['review'][index])
        for noun in nouns:
            if len(noun) > 3 and 'thing' not in noun:
                new_term.append(noun)
        sf = sf.append({'id': df['sentenceID'][index], 
                    'target': df['target'][index],
                    'review': df['review'][index],
                    'category': df['category'][index],
                    'term': '|'.join(new_term)
                   }, ignore_index=True)

    sf.to_csv("Results/Aspect Terms Extraction/16_nan.csv")
    sf.to_excel("Results/Aspect Terms Extraction/16_nan.xlsx")

In [7]:
chunk('dataset/res16_filter_nan.csv')

HBox(children=(IntProgress(value=0, max=1104), HTML(value='')))

err :  Not because you are "The Four Seasons"... – you are allowed to charge an arm and a leg for a romatic dinner.
err :  Gross food – Wow-
err :  short and sweet – seating is great:it's romantic,cozy and private.
err :  i love margherita pizza – i looove east village pizza
err :  This is undoubtedly my favorite modern Japanese brasserie (that don’t serve sushi), and in my opinion, one of the most romantic restaurants in the city!
err :  Not only is it an adventure getting to this somewhat hidden spot, once you enter the unmarked wooden doors, the zen and intimate décor will make you feel like you’re no longer in the city.
err :  If you’re planning to come here, make sure that your date is someone whom you really like since you’ll be ushered to private booths where there will be no people or food watching (choose the ones on the ground level that have glass ceilings so you may see the stars in the sky!).
err :  It’s just you and your date and an occasional cute ‘excuse me’ before the 